In [1]:
import keras
from keras.applications import vgg16
from keras.models import Model
from keras.layers import Input, Flatten, Dense
import numpy as np

Using Theano backend.
/home/hyz/.virtualenvs/myenv2/local/lib/python2.7/site-packages/theano/gpuarray/dnn.py:135: UserWarning: Your cuDNN version is more recent than Theano. If you encounter problems, try updating Theano or downgrading cuDNN to version 5.1.
  warnings.warn("Your cuDNN version is more recent than "
Using cuDNN version 7001 on context None
Mapped name None to device cuda0: GeForce GTX 960M (0000:01:00.0)


In [2]:
model = vgg16.VGG16(weights='imagenet', include_top=False)
for l in model.layers:
    l.trainable=False

In [3]:
model.layers

In [4]:
input = Input(shape=(32,32,3), name='image_input')
output_vgg16_conv = model(input)

x = Flatten(name='flatten')(output_vgg16_conv)
x = Dense(4096, activation='relu', name='fc1')(x)
x = Dense(4096, activation='relu', name='fc2')(x)
x = Dense(10, activation='softmax', name='predictions')(x)

#Create your own model 
my_model = Model(input=input, output=x)

/home/hyz/.virtualenvs/myenv2/lib/python2.7/site-packages/ipykernel_launcher.py:10: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Softmax.0, inputs=/image_inp...)`
  # Remove the CWD from sys.path while we load stuff.


In [5]:
my_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
image_input (InputLayer)     (None, 32, 32, 3)         0         
_________________________________________________________________
vgg16 (Model)                multiple                  14714688  
_________________________________________________________________
flatten (Flatten)            (None, 512)               0         
_________________________________________________________________
fc1 (Dense)                  (None, 4096)              2101248   
_________________________________________________________________
fc2 (Dense)                  (None, 4096)              16781312  
_________________________________________________________________
predictions (Dense)          (None, 10)                40970     
Total params: 33,638,218
Trainable params: 18,923,530
Non-trainable params: 14,714,688
_______________________________________________________

In [6]:
import scipy.io as sio

data = sio.loadmat('/home/hyz/prog/deeplearning/housenumbers/train_32x32.mat')

x_train = data['X']
y_train = data['y']
print x_train.shape, y_train.shape

y_train = y_train.reshape(y_train.shape[0]) - 1

img_rows = 32
img_cols = 32
print x_train.shape, y_train.shape
input_shape = (img_rows, img_cols, 3)
x2 = np.array([x_train[:,:,:,i] for i in range(x_train.shape[-1])])
x_train = x2
num_classes = 10
y_train = keras.utils.to_categorical(y_train, num_classes)

opt = keras.optimizers.Adadelta(lr=0.001)
my_model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])
print my_model.summary()


(32, 32, 3, 73257) (73257, 1)
(32, 32, 3, 73257) (73257,)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
image_input (InputLayer)     (None, 32, 32, 3)         0         
_________________________________________________________________
vgg16 (Model)                multiple                  14714688  
_________________________________________________________________
flatten (Flatten)            (None, 512)               0         
_________________________________________________________________
fc1 (Dense)                  (None, 4096)              2101248   
_________________________________________________________________
fc2 (Dense)                  (None, 4096)              16781312  
_________________________________________________________________
predictions (Dense)          (None, 10)                40970     
Total params: 33,638,218
Trainable params: 18,923,530
Non-trainable params: 14,714,6

In [7]:
my_model.fit(x_train, y_train,
             batch_size=128, epochs=10, verbose=1, validation_split=0.1,
             shuffle=True)


Train on 65931 samples, validate on 7326 samples
Epoch 1/10
65931/65931 [==============================] - 76s - loss: 2.9654 - acc: 0.1994 - val_loss: 2.5466 - val_acc: 0.2536
Epoch 2/10
65931/65931 [==============================] - 77s - loss: 2.3477 - acc: 0.2903 - val_loss: 2.2547 - val_acc: 0.3093
Epoch 3/10
65931/65931 [==============================] - 78s - loss: 2.1348 - acc: 0.3374 - val_loss: 2.1111 - val_acc: 0.3392
Epoch 4/10
65931/65931 [==============================] - 80s - loss: 2.0110 - acc: 0.3659 - val_loss: 2.0170 - val_acc: 0.3564
Epoch 5/10
65931/65931 [==============================] - 80s - loss: 1.9251 - acc: 0.3873 - val_loss: 1.9482 - val_acc: 0.3699
Epoch 6/10
65931/65931 [==============================] - 81s - loss: 1.8598 - acc: 0.4050 - val_loss: 1.8940 - val_acc: 0.3832
Epoch 7/10
65931/65931 [==============================] - 82s - loss: 1.8069 - acc: 0.4200 - val_loss: 1.8515 - val_acc: 0.4002
Epoch 8/10
65931/65931 [==============================]

In [8]:
dir(keras.applications)

['InceptionV3',
 'MobileNet',
 'ResNet50',
 'VGG16',
 'VGG19',
 'Xception',
 '__builtins__',
 '__doc__',
 '__file__',
 '__name__',
 '__package__',
 '__path__',
 'imagenet_utils',
 'inception_v3',
 'mobilenet',
 'resnet50',
 'vgg16',
 'vgg19',
 'xception']